In [ ]:
import pickle
from pathlib import Path
import pandas as pd

def read_data(file_path):
    with open(file_path, 'rb') as f:
        list = pickle.load(f)
    return list

In [ ]:
prefix = Path('validation_res/')

detector = prefix / 'time_agnostic_detector' / 'times.pickle'
unpruned = prefix / 'time_classification_unpruned' / 'times.pickle'
pruned =  prefix / 'time_classification_pruning' / 'times.pickle'
kd = prefix / 'time_classification_kd' / 'times.pickle'

detector_dict = read_data(detector)
unpruned_dict = read_data(unpruned)
pruned_dict = read_data(pruned)
kd_dict = read_data(kd)

In [ ]:
detector_dict

In [ ]:
detector_df = pd.DataFrame(detector_dict)
detector_df['method'] = ['Detector'] * len(detector_df)

unpruned_df = pd.DataFrame(unpruned_dict)
unpruned_df['method'] = ['Detector<br>+<br>CIL Unpruned'] * len(unpruned_df)

pruned_df = pd.DataFrame(pruned_dict)
pruned_df['method'] = ['Detector<br>+<br>CIL Pruned'] * len(pruned_df)

kd_df = pd.DataFrame(kd_dict)
kd_df['method'] = ['Detector<br>+<br>CIL KD'] * len(kd_df)

final_df = pd.concat([detector_df, unpruned_df, pruned_df, kd_df])
final_df['cil_inference'] = final_df['cil_inference'].fillna(0)

final_df.head()


In [ ]:
df_numeric = final_df[['convert_img', 'yolo_inference', 'non_max_supp', 'cil_inference']]
final_df['total'] = df_numeric.sum(axis=1)
final_df.head()

In [ ]:
final_df_agg = final_df.groupby(['method']).agg(['mean', 'std'])
final_df_agg

In [ ]:
import plotly.express as px

from scipy import stats
import numpy as np


final_df_clean = final_df[(np.abs(stats.zscore(final_df['total'])) < 2)]
fig = px.box(final_df_clean, x="method", y='total', color='method', template='seaborn')
fig.update_xaxes(title='Method')
fig.update_yaxes(title='Time (s)')
fig.update_layout(showlegend=False)

fig.update_layout(
    title={
        'text': "Inference time of models (CPU)",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_image('times.png', scale=3)

In [ ]:
final_df[(np.abs(stats.zscore(final_df['total'])) < 3)]